[Gantt with plotly](https://plotly.com/python/gantt/)
* for loop on author, create additional dict
* if author has fixed dates, one entry (param "sure")
* if author has uncertain dates, one entry for uncertain birth span (param "uncertain"), one entry for sure life (param "sure"), one entry for uncertain death span (param "uncertain")
* param "sure" = full bar - param "uncertain" = stripes
* color according to total number of mentions?